Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

# Importing Libraries

In [5]:
#Importing Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Creating the Dataset

Getting data from wikipedia

In [6]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

# Using Soup to better analyze it

In [7]:
#USING THE SOUP
# find the table
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [8]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [9]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Modifying the data making it looks exacly equals to the project

In [10]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Creating a test Dataframe

In [13]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


# Making the last print of the size of the dataframe

In [14]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

# Starting Part 2

# Creating the Geospatial Data

In [31]:
Geospatial_code = [["M1B","43.8066863","-79.1943534"],["M1C","43.7845351","-79.1604971"],["M1E","43.7635726","-79.1887115"],["M1G","43.7709921","-79.2169174"],["M1H","43.773136","-79.2394761"],["M1J","43.7447342","-79.2394761"],["M1K","43.7279292","-79.2620294"],["M1L","43.7111117","-79.2845772"],["M1M","43.716316","-79.2394761"],["M1N","43.692657","-79.2648481"],["M1P","43.7574096","-79.273304"],["M1R","43.7500715","-79.2958491"],["M1S","43.7942003","-79.2620294"],["M1T","43.7816375","-79.3043021"],["M1V","43.8152522","-79.2845772"],["M1W","43.7995252","-79.3183887"],["M1X","43.8361247","-79.2056361"],["M2H","43.8037622","-79.3634517"],["M2J","43.7785175","-79.3465557"],["M2K","43.7869473","-79.385975"],["M2L","43.7574902","-79.3747141"],["M2M","43.789053","-79.4084928"],["M2N","43.7701199","-79.4084928"],["M2P","43.7527583","-79.4000493"],["M2R","43.7827364","-79.4422593"],["M3A","43.7532586","-79.3296565"],["M3B","43.7459058","-79.352188"],["M3C","43.7258997","-79.340923"],["M3H","43.7543283","-79.4422593"],["M3J","43.7679803","-79.4872619"],["M3K","43.7374732","-79.4647633"],["M3L","43.7390146","-79.5069436"],["M3M","43.7284964","-79.4956974"],["M3N","43.7616313","-79.5209994"],["M4A","43.7258823","-79.3155716"],["M4B","43.7063972","-79.309937"],["M4C","43.6953439","-79.3183887"],["M4E","43.6763574","-79.2930312"],["M4G","43.7090604","-79.3634517"],["M4H","43.7053689","-79.3493719"],["M4J","43.685347","-79.3381065"],["M4K","43.6795571","-79.352188"],["M4L","43.6689985","-79.3155716"],["M4M","43.6595255","-79.340923"],["M4N","43.7280205","-79.3887901"],["M4P","43.7127511","-79.3901975"],["M4R","43.7153834","-79.4056784"],["M4S","43.7043244","-79.3887901"],["M4T","43.6895743","-79.3831599"],["M4V","43.6864123","-79.4000493"],["M4W","43.6795626","-79.3775294"],["M4X","43.667967","-79.3676753"],["M4Y","43.6658599","-79.3831599"],["M5A","43.6542599","-79.3606359"],["M5B","43.6571618","-79.3789371"],["M5C","43.6514939","-79.3754179"],["M5E","43.6447708","-79.3733064"],["M5G","43.6579524","-79.3873826"],["M5H","43.6505712","-79.3845675"],["M5J","43.6408157","-79.3817523"],["M5K","43.6471768","-79.3815764"],["M5L","43.6481985","-79.3798169"],["M5M","43.7332825","-79.4197497"],["M5N","43.7116948","-79.4169356"],["M5P","43.6969476","-79.4113072"],["M5R","43.6727097","-79.4056784"],["M5S","43.6626956","-79.4000493"],["M5T","43.6532057","-79.4000493"],["M5V","43.6289467","-79.3944199"],["M5W","43.6464352","-79.374846"],["M5X","43.6484292","-79.3822802"],["M6A","43.718518","-79.4647633"],["M6B","43.709577","-79.4450726"],["M6C","43.6937813","-79.4281914"],["M6E","43.6890256","-79.453512"],["M6G","43.669542","-79.4225637"],["M6H","43.6690051","-79.4422593"],["M6J","43.6479267","-79.4197497"],["M6K","43.6368472","-79.4281914"],["M6L","43.7137562","-79.4900738"],["M6M","43.6911158","-79.4760133"],["M6N","43.6731853","-79.4872619"],["M6P","43.6616083","-79.4647633"],["M6R","43.6489597","-79.456325"],["M6S","43.6515706","-79.4844499"],["M7A","43.6623015","-79.3894938"],["M7R","43.6369656","-79.615819"],["M7Y","43.6627439","-79.321558"],["M8V","43.6056466","-79.5013207"],["M8W","43.6024137","-79.5434841"],["M8X","43.6536536","-79.5069436"],["M8Y","43.6362579","-79.4985091"],["M8Z","43.6288408","-79.5209994"],["M9A","43.6678556","-79.5322424"],["M9B","43.6509432","-79.5547244"],["M9C","43.6435152","-79.5772008"],["M9L","43.7563033","-79.5659633"],["M9M","43.7247659","-79.5322424"],["M9N","43.706876","-79.5181884"],["M9P","43.696319","-79.5322424"],["M9R","43.6889054","-79.5547244"],["M9V","43.7394164","-79.5884369"],["M9W","43.7067483","-79.5940544"]]

In [39]:
from pandas import DataFrame
coordinates = DataFrame.from_records(Geospatial_code)
coordinates.columns = ["Postal Code","Latitude","Longitude"]

In [40]:
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


In [41]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.8066863,-79.1943534
1,M1C,43.7845351,-79.1604971
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.2169174
4,M1H,43.773136,-79.2394761


In [42]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


In [43]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.6579524,-79.3873826
1,M2H,North York,Hillcrest Village,43.8037622,-79.3634517
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7063972,-79.309937
3,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
4,M4G,East York,Leaside,43.7090604,-79.3634517
5,M4M,East Toronto,Studio District,43.6595255,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.7500715,-79.2958491
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.7394164,-79.5884369
8,M9L,North York,Humber Summit,43.7563033,-79.5659633
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.6289467,-79.3944199
